In [51]:
import mysql.connector #Connects to the MySQL database to store user mistakes
import openai #Used for interacting with OpenAI's GPT model
from langchain.memory import ConversationBufferMemory #Maintains chat history for better context tracking
import json

In [3]:
# Database Setup
def connect_to_db():
    try:
        db = mysql.connector.connect(
            host="localhost",
            user="Your Username",
            password="Your Password",
            database="language_bot",
            connection_timeout=600  # Increased timeout to prevent disconnection issues
        )
        return db, db.cursor()
    except mysql.connector.Error as err:
        print(f"Database connection error: {err}")
        exit()

db, cursor = connect_to_db()

cursor.execute("""
    CREATE TABLE IF NOT EXISTS mistakes (
        id INT AUTO_INCREMENT PRIMARY KEY,
        user_id VARCHAR(255),
        mistake TEXT,
        correction TEXT,
        timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    )
""")
db.commit()

In [33]:
# OpenAI API Setup
OPENAI_API_KEY = "Your API Toekn Key"
openai.api_key = OPENAI_API_KEY

In [35]:
# Initialize LangChain Memory
session_memories = {}

def get_session_memory(session_id):
    if session_id not in session_memories:
        session_memories[session_id] = ConversationBufferMemory(return_messages=True)
    return session_memories[session_id]

In [49]:
# The chatbot processes user input and generates a response using OpenAI’s API.
def invoke_chatbot(user_input, session_id):
    memory = get_session_memory(session_id)
    memory.save_context({"human": user_input}, {"ai": "Processing..."})
    try:
        client = openai.OpenAI(api_key=OPENAI_API_KEY)
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a language tutor. Engage in conversation while gently correcting mistakes."},
                {"role": "user", "content": user_input}
            ]
        )
        reply = response.choices[0].message.content
        memory.save_context({"human": user_input}, {"ai": reply})
        return reply
    except Exception as e:
        return f"Error communicating with the model: {str(e)}"

In [41]:
def find_differences(original, corrected):
    original_words = original.split()
    corrected_words = corrected.split()
    mistakes = {}
    
    for o, c in zip(original_words, corrected_words):
        if o != c:
            mistakes[o] = c
    
    return mistakes

In [43]:
def save_mistake(user_id, mistake, correction):
    cursor.execute("""
        INSERT INTO mistakes (user_id, mistake, correction)
        VALUES (%s, %s, %s)
    """, (user_id, mistake, correction))
    db.commit()

In [45]:
def get_mistakes_summary(user_id):
    cursor.execute("""
        SELECT mistake, correction FROM mistakes WHERE user_id = %s
    """, (user_id,))
    return cursor.fetchall()

In [47]:
def chatbot():
    user_id = input("Enter your user ID: ")
    known_lang = input("What language do you know? ")
    target_lang = input("What language do you want to learn? ")
    proficiency = input("What is your proficiency level (beginner/intermediate/advanced)? ")
    session_id = user_id  # Assigning user_id as session_id
    
    print("Starting conversation in", target_lang, "...")
    print("Type 'exit' to stop the chat or 'review' to see your mistakes.")
    
    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        elif user_input.lower() == "review":
            print("Reviewing your mistakes...")
            mistakes = get_mistakes_summary(user_id)
            for mistake, correction in mistakes:
                print(f"Mistake: {mistake} -> Correction: {correction}")
            continue

        corrected_text = invoke_chatbot(user_input, session_id)
        mistakes_dict = find_differences(user_input, corrected_text)
        
        for mistake, correction in mistakes_dict.items():
            save_mistake(user_id, mistake, correction)
        
        print(f"Chatbot: {corrected_text}")

    print("Chatbot session ended.")

if __name__ == "__main__":
    chatbot()

Enter your user ID:  gaurav
What language do you know?  hindi
What language do you want to learn?  english
What is your proficiency level (beginner/intermediate/advanced)?  beginner


Starting conversation in english ...
Type 'exit' to stop the chat or 'review' to see your mistakes.


You:  hello


Chatbot: Error communicating with the model: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}


You:  review


Reviewing your mistakes...
Mistake: hello -> Correction: Error


You:  exit


Chatbot session ended.
